In [1]:
from simpleai.search import SearchProblem, astar, breadth_first, depth_first, uniform_cost
from simpleai.search.viewers import BaseViewer

from tqdm import tqdm

PSA de Busqueda Informada utilizando el algoritmo A*

In [2]:
class Cleanup(SearchProblem):
    def __init__(self, initial, goal):
        super().__init__(initial_state = initial)
        self.goal_state = goal
        self.visited_states = {}

    def actions(self, state):
        
        valid_actions = []
        for i in range(len(state)):
            for j in range(len(state[0])):
                valid_actions.append((i, j))
        return valid_actions
    
    def result(self, state, action):
        i, j = action
        child_state = [list(row) for row in state]

        if child_state in self.visited_states:
            return None

        if i - 1 >= 0:
            child_state[i - 1][j] = not child_state[i - 1][j]
        if j - 1 >= 0:
            child_state[i][j - 1] = not child_state[i][j - 1]
        if j + 1 < len(state[0]):
            child_state[i][j + 1] = not child_state[i][j + 1]
        if i + 1 < len(state):
            child_state[i + 1][j] = not child_state[i + 1][j]

        return tuple([tuple(row) for row in child_state])
    
    def is_goal(self, state):
        return state == self.goal_state
    
    def cost(self, state, action, state2):
        return 1
    
    def heuristic(self, state):
        sum(state)
        return sum(state)

In [3]:
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i, action)
                print('¡Meta lograda con costo = ', result.cost,'!')
            else:
                print(i, action)

            print('  ', state)
    else:
        print('Mala configuración del problema')

In [4]:
# init = ((False, False, True, False, False),
#         (False, True, False, True, False),
#         (False, False, True, False, False),
#         (False, False, False, False, False),
#         (False, False, False, False, False))

# goal = ((False, False, False, False, False),
#         (False, False, False, False, False),
#         (False, False, False, False, False),
#         (False, False, False, False, False),
#         (False, False, False, False, False))

init = ((True, True, True, False, False, False, False, False, False, False, False),
        (True, True, False, True, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False))

goal = ((False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False),
        (False, False, False, False, False, False, False, False, False, False, False))


In [5]:
my_viewer = BaseViewer()       # Solo estadísticas

# result = astar(Cleanup(init,goal), graph_search=True)
result = breadth_first(Cleanup(init,goal), graph_search=True)
# result = depth_first(Cleanup(init,goal), graph_search=True)
# result = uniform_cost(Cleanup(init,goal), graph_search=True)

if my_viewer != None:
    print('Stats:')
    print(my_viewer.stats)

print()
print('>> Búsqueda A* <<')
display(result)


TypeError: unhashable type: 'list'